In [ ]:
"""
Probe testing
"""
None

In [ ]:
"""
Imports
"""
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, Glm4vForConditionalGeneration, AutoModelForImageTextToText
from transformers.loss.loss_utils import ForCausalLMLoss
from datasets import load_dataset
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import gc
import pickle
import textwrap
from termcolor import colored
import importlib
import cupy
import cuml
import sklearn

from utils.memory import check_memory, clear_all_cuda_memory
from utils.store_outputs import convert_outputs_to_df_fast

main_device = 'cuda:0'
seed = 123

ws = '/workspace/deliberative-alignment-jailbreaks'

clear_all_cuda_memory()
check_memory()

# Load model

In [ ]:
"""
Load the base tokenizer/model
"""
selected_model_index = 0

def get_model(index):
    # HF model ID, model prefix, model architecture,  attn implementation, whether to use hf lib implementation, is reasoning
    models = {
        0: ('openai/gpt-oss-20b', 'gptoss20', 'gptoss', 'kernels-community/vllm-flash-attn3', True, 24, True),
        1: ('openai/gpt-oss-120b', 'gptoss120', 'gptoss', 'kernels-community/vllm-flash-attn3', True, 36, True), # Will load experts in MXFP4 if triton kernels installed
        2: ('allenai/Olmo-3-7B-Think', 'olmo3-7', 'olmo3', None, True, 32, True), # OlMo-3
        3: ('allenai/Olmo-3-32B-Think', 'olmo3-32', 'olmo3', None, True, 64, True),
        4: ('zai-org/GLM-4.6V-Flash', 'glm-46v-flash', 'glm46v', None, True, 40, True),
        5: ('ServiceNow-AI/Apriel-1.6-15b-Thinker', 'apriel-16', 'apriel', None, True, 48, True),
        6: ('mistralai/Devstral-Small-2-24B-Instruct-2512', 'mistral3', 'devstral', None, True, 40, False)
    }
    return models[index]

def load_model_and_tokenizer(model_id, model_prefix, model_attn, model_use_hf):
    """
    Load the model and tokenizer from HF, or from file if already downloaded.
    """
    cache_dir = '/workspace/hf'
    tokenizer = AutoTokenizer.from_pretrained(model_id, cache_dir = cache_dir, add_eos_token = False, add_bos_token = False, padding_side = 'left', trust_remote_code = True)
    load_params = {'cache_dir': cache_dir, 'dtype': 'auto', 'trust_remote_code': not model_use_hf, 'device_map': None, 'attn_implementation': model_attn}    
    if model_architecture == 'glm46v':
        model = Glm4vForConditionalGeneration.from_pretrained(model_id, **load_params).to(main_device).eval()
    elif model_architecture == 'apriel':
        model = AutoModelForImageTextToText.from_pretrained(model_id, **load_params).to(main_device).eval()
    else:
        model = AutoModelForCausalLM.from_pretrained(model_id, **load_params).to(main_device).eval()

    return tokenizer, model

model_id, model_prefix, model_architecture, model_attn, model_use_hf, model_n_layers, model_is_reasoning = get_model(selected_model_index)
tokenizer, model = load_model_and_tokenizer(model_id, model_prefix, model_attn, model_use_hf)

# Check attention + exper precision for gpt-oss-*
if model_architecture == 'gptoss':
    print(model.model.layers[0].mlp.experts.down_proj)
    print(model.model.config._attn_implementation)

check_memory()

In [ ]:
"""
Load reverse-engineered forward pass functions (note for other users - this can be replaced by simpler hooks if desired; we use custom functions due to intersection of other research projects)
"""
model_module = importlib.import_module(f"utils.pretrained_models.{model_architecture}")
run_model_return_topk = getattr(model_module, f"run_{model_architecture}_return_topk")

@torch.no_grad()
def test_custom_forward_pass(model, pad_token_id):
    inputs = tokenizer(['Hi! I am a dog and I like to bark', 'Vegetables are good for'], return_tensors = 'pt', padding = 'max_length', truncation = True, max_length = 640).to(model.device)
    original_results = model(**inputs, use_cache = False)
    custom_results = run_model_return_topk(model, inputs['input_ids'], inputs['attention_mask'], return_hidden_states = True)
    assert torch.equal(original_results.logits, custom_results['logits']), 'Error in custom forward'
    loss = ForCausalLMLoss(custom_results['logits'], torch.where(inputs['input_ids'] == pad_token_id, torch.tensor(-100), inputs['input_ids']), custom_results['logits'].size(-1)).detach().cpu().item()
    print(f"LM loss: {loss}")
    print(f"Hidden states layers (pre-mlp | post-layer): {len(custom_results['all_pre_mlp_hidden_states'])} | {len(custom_results['all_hidden_states'])}")
    print(f"Hidden state size (pre-mlp | post-layer): {(custom_results['all_pre_mlp_hidden_states'][0].shape)} | {(custom_results['all_hidden_states'][0].shape)}")

test_custom_forward_pass(model, tokenizer.pad_token_id)

# Train probes

## Prep dataset

In [ ]:
"""
Load dataset - C4 + Dolma3
"""
n_sample_size = 250 # 500 default + tested

def load_raw_ds():

    def get_c4():
        return load_dataset('allenai/c4', 'en', split = 'validation', streaming = True).shuffle(seed = seed, buffer_size = 50_000)
    
    def get_dolma3():
        return load_dataset('allenai/dolma3_mix-150B-1025', split = 'train', streaming = True).shuffle(seed = seed, buffer_size = 10_000)
    
    # def get_hplt2():
    #     return load_dataset('HPLT/HPLT2.0_cleaned', 'eng_Latn', split = 'train', streaming = True).shuffle(seed = seed, buffer_size = 50_000)

    def get_data(ds, n_samples, data_source): # en
        raw_data = []
        ds_iter = iter(ds)
        for _ in range(n_samples):
            sample = next(ds_iter, None)
            if sample is None:
                break
            raw_data.append({'text': sample['text'], 'source': data_source})
        return raw_data
    
    return get_data(get_c4(), int(n_sample_size * .2), 'c4-en') + get_data(get_dolma3(), int(n_sample_size * .8), 'dolma3')

raw_data = load_raw_ds()

In [ ]:
"""
Test our formattting functions
"""
import utils.role_templates
importlib.reload(utils.role_templates)
from utils.role_templates import render_single_message, render_mixed_cot

# Test
print(tokenizer.apply_chat_template(
    [
        {'role': 'user', 'content': 'Hi! I am a dog and I like to bark'},
        {'role': 'assistant', 'content': 'Hello! What a lovely dog you are!'}
    ],
    tokenize = False, padding = 'max_length', truncation = True, max_length = 512, add_generation_prompt = False
))

print('--')
print(render_single_message(model_architecture, role = 'assistant', content ='Hi'))
print('--')
print(render_mixed_cot(model_architecture, 'The user...', 'Yes!'))
print('--')

In [ ]:
"""
Define opening text to append at the start of each role
"""
if model_architecture == 'gptoss':
    train_prefixes = [""]

elif model_architecture == 'olmo3':
    train_prefixes = [
        tokenizer.bos_token + render_single_message(
            model_architecture,
            'system', 
            textwrap.dedent(
                """
                You are OLMo, a helpful function-calling AI assistant built by Ai2. Your date cutoff is November 2024, and your model weights are available at https://huggingface.co/allenai. You do not currently have access to any functions. <functions></functions>
                """
            ).strip()
        )
    ]
    
elif model_architecture in ['glm46v', 'glm4vmoe']:
    train_prefixes = ["""[gMASK]<sop>"""]

elif model_architecture == 'apriel':
    train_prefixes = [
        tokenizer.bos_token + render_single_message(
            model_architecture,
            'system',
            textwrap.dedent(
                """
                You are a thoughtful, systematic AI assistant from ServiceNow Language Models (SLAM) lab. Analyze each question carefully, present your reasoning step-by-step, then provide the final response after the marker [BEGIN FINAL RESPONSE].
                """
            ).strip()
        )
    ]

else:
    raise ValueError('Missing model architecture')

for p in train_prefixes:
    print(p)

In [ ]:
"""
Run some IFT tests
"""
import utils.role_templates
importlib.reload(utils.role_templates)
from utils.role_templates import render_single_message, render_mixed_cot

# Test
print(tokenizer.apply_chat_template(
    [
        {'role': 'user', 'content': 'Hi! I am a dog and I like to bark'},
        {'role': 'assistant', 'content': 'Hello! What a lovely dog you are!'}
    ],
    tokenize = False, padding = 'max_length', truncation = True, max_length = 512, add_generation_prompt = False
))

print('--')
print(render_single_message(model_architecture, role = 'assistant', content ='Hi'))
print('--')
print(render_mixed_cot(model_architecture, 'The user...', 'Yes!'))
print('--')

In [ ]:
"""
Create sample sequences
"""
SEQLEN = 1536
IS_MERGE = False # Whether to prefix random CoT before assistant text - do for models with nested <think></think>
IS_REASONING = model_is_reasoning

def get_sample_seqs(probe_text, partner_text, prefix = ''):
    """
    Params
        @probe_text: The text we're extracting states from (appears in all roles)
        @partner_text: Random paired text (only used in merged sample's assistant position)
    """
    seqs = []
    for role in ['system', 'user', 'tool']:
        seqs.append({
            'role': role,
            'prompt': prefix + render_single_message(model_architecture, role = role, content = probe_text)
        })

    if IS_REASONING:
        seqs.append({
            'role': 'cot',
            'prompt': prefix + render_single_message(model_architecture, role = 'cot', content = probe_text)
        })

    # If merge + LRM, render with CoT prefix
    if IS_REASONING and IS_MERGE:
        seqs.append({
            'role': 'assistant',
            'prompt': prefix + render_mixed_cot(model_architecture, cot = partner_text, assistant = probe_text)
        })
    # Else render standalone
    else:
        seqs.append({
            'role': 'assistant',
            'prompt': prefix + render_single_message(model_architecture, role = 'assistant', content = probe_text)
        })

    return seqs


truncated_texts = tokenizer.batch_decode(tokenizer([t['text'] for t in raw_data], padding = False, truncation = True, max_length = SEQLEN).input_ids)
n_seqs = len(truncated_texts)

np.random.seed(seed)
partner_lengths = np.rint(np.random.lognormal(mean = 1.5, sigma = 2, size = n_seqs)).astype(int).tolist() # Reasonable seq length sampling
partner_texts = [
    tokenizer.decode(tokenizer(text['text'], padding = False, truncation = True, max_length = int(partner_lengths[i])).input_ids)
    for i, text in enumerate(raw_data)
]
perm = np.random.permutation(n_seqs)
while np.any(perm == np.arange(n_seqs)):
    perm = np.random.permutation(n_seqs)

sampled_prefixes = np.random.choice(train_prefixes, size = n_seqs)

input_list = []
for base_ix, base_text in enumerate(truncated_texts):
    partner_text = partner_texts[int(perm[base_ix])]
    prefix = sampled_prefixes[base_ix]

    for seq in get_sample_seqs(base_text, partner_text, prefix):
        row = {'question_ix': base_ix, 'question': base_text, **seq}
        input_list.append(row)

input_df = pd.DataFrame(input_list).assign(prompt_ix = lambda df: list(range(len(df))))
display(input_df)

# Print examples for simple ex
for p in [row['prompt'] for row in input_df.pipe(lambda df: df[df['question_ix'] == 1]).to_dict('records')]:
    print(p)
    print("=" * 80)

In [ ]:
""" 
Load dataset into a dataloader. The dataloader returns the original tokens - this is important for BPE tokenizers as otherwise it's difficult to reconstruct the correct string later!
"""
from utils.dataset import ReconstructableTextDataset, stack_collate
from torch.utils.data import DataLoader

max_seqlen = int(tokenizer(input_df['prompt'].tolist(), padding = True, truncation = False, return_tensors = 'pt')['attention_mask'].sum(dim = 1).max().item())
train_dl = DataLoader(
    ReconstructableTextDataset(input_df['prompt'].tolist(), tokenizer, max_length = max_seqlen, prompt_ix = input_df['prompt_ix'].tolist()),
    batch_size = 16,
    shuffle = False,
    collate_fn = stack_collate
)

## Get hidden states

In [ ]:
"""
Run forward passes
"""
@torch.no_grad()
def run_and_export_states(model, dl: ReconstructableTextDataset, layers_to_keep_acts: list[int]):
    """
    Run forward passes on given model and store the decomposed sample_df plus hidden states

    Params:
        @model: The model to run forward passes on. Should return a dict with keys `logits` and `all_hidden_states`.
        @dl: A ReconstructableTextDataset of which returns `input_ids`, `attention_mask`, and `original_tokens`.
        @layers_to_keep_acts: A list of layer indices (0-indexed) for which to filter `all_hidden_states` (see returned object description).

    Returns:
        A dict with keys:
        - `sample_df`: A sample (token)-level dataframe with corresponding input token ID, output token ID, and input token text (removes masked tokens)
        - `all_hidden_states`: A tensor of size n_samples x layers_to_keep_acts x D return the hidden state for each retained layers. Each 
            n_sample corresponds to a row of sample_df.

    Example:
        test_outputs = run_and_export_states(model, train_dl, layers_to_keep_acts = list(range(model_n_layers)))
    """
    all_hidden_states = []
    sample_dfs = []

    for batch_ix, batch in tqdm(enumerate(dl), total = len(dl)):

        input_ids = batch['input_ids'].to(main_device)
        attention_mask = batch['attention_mask'].to(main_device)
        original_tokens = batch['original_tokens']
        prompt_indices = batch['prompt_ix']

        output = run_model_return_topk(model, input_ids, attention_mask, return_hidden_states = True)

        # Check no bugs by validating output/perplexity
        if batch_ix == 0:
            loss = ForCausalLMLoss(output['logits'], torch.where(input_ids == tokenizer.pad_token_id, torch.tensor(-100), input_ids), output['logits'].size(-1)).detach().cpu().item()
            for i in range(min(20, input_ids.size(0))):
                decoded_input = tokenizer.decode(input_ids[i, :], skip_special_tokens = False)
                next_token_id = torch.argmax(output['logits'][i, -1, :]).item()
                print('---------\n' + decoded_input + colored(tokenizer.decode([next_token_id], skip_special_tokens = False).replace('\n', '<lb>'), 'green'))
            print(f"PPL:", torch.exp(torch.tensor(loss)).item())
                
        original_tokens_df = pd.DataFrame(
            [(seq_i, tok_i, tok) for seq_i, tokens in enumerate(original_tokens) for tok_i, tok in enumerate(tokens)], 
            columns = ['sequence_ix', 'token_ix', 'token']
        )
                
        prompt_indices_df = pd.DataFrame(
            [(seq_i, seq_source) for seq_i, seq_source in enumerate(prompt_indices)], 
            columns = ['sequence_ix', 'prompt_ix']
        )
        
        # Create sample (token) level dataframe
        sample_df =\
            convert_outputs_to_df_fast(input_ids, attention_mask, output['logits'])\
            .merge(original_tokens_df, how = 'left', on = ['token_ix', 'sequence_ix'])\
            .merge(prompt_indices_df, how = 'left', on = ['sequence_ix'])\
            .assign(batch_ix = batch_ix)
        
        sample_dfs.append(sample_df)

        # Store pre-MLP hidden states - the fwd pass as n_layers list as BN x D, collapse to BN x n_layers x D, with BN filtering out masked items
        valid_pos = torch.where(attention_mask.cpu().view(-1) == 1) # Valid (BN, ) positions
        all_hidden_states.append(torch.stack(output['all_pre_mlp_hidden_states'], dim = 1)[valid_pos][:, layers_to_keep_acts, :])

    sample_df = pd.concat(sample_dfs, ignore_index = True).drop(columns = ['batch_ix', 'sequence_ix']) # Drop batch/seq_ix, since prompt_ix identifies
    all_hidden_states = torch.cat(all_hidden_states, dim = 0)

    return {
        'sample_df': sample_df,
        'all_hs': all_hidden_states
    }

layers_to_probe = list(range(0, model_n_layers, 4 if model_n_layers >= 40 else 4))
res = run_and_export_states(model, train_dl, layers_to_keep_acts = layers_to_probe)

## Label roles


In [ ]:
"""
Label roles
"""
import utils.role_assignments
importlib.reload(utils.role_assignments)
from utils.role_assignments import label_content_roles
from utils.substring_assignments import flag_message_types

probe_sample_df = (
    # Flag all roles
    label_content_roles(model_architecture, res['sample_df'])    
    .assign(sample_ix = lambda df: range(0, len(df)))
    # Flag those that match target role (for retain later)
    .merge(input_df[['prompt_ix', 'role']].rename(columns = {'role': 'target_role'}), how = 'inner', on = 'prompt_ix')
    .assign(match_target_role = lambda df: np.where(df['role'] == df['target_role'], True, False))
    # Flag those that match the prepend (for drop later)
    .pipe(lambda df: flag_message_types(df, train_prefixes))
    .drop(columns = 'base_message')
    # Drop prepend text + non-content tags
    .pipe(lambda df: df[(df['base_message_ix'].isna()) & (df['is_content'] == True) & (df['role'].notna()) & (df['match_target_role']) ])
)

# Verify role counts are accurate (should be exactly equal for most models, slight discrepancy for Olmo3 since <think> isn't a single tok)
display(
    probe_sample_df\
    .groupby('role', as_index = False)\
    .agg(count = ('sample_ix', 'count'))
)

# Validate roles are flagged correctly
display(
    probe_sample_df\
    .pipe(lambda df: df[df['prompt_ix'] <= 14])\
    .groupby(['prompt_ix', 'seg_ix', 'role'], as_index = False)\
    .agg(combined_text = ('token', ''.join))\
    .assign(eot = lambda df: df['combined_text'].str[-30:])
)

# probe_sample_df.pipe(lambda df: df[df['prompt_ix'] == 3]).to_csv('dump.csv')

In [ ]:
"""
Convert activations to fp16 (for compatibility with cupy later) + layer-wise dict
"""
all_probe_hs = res['all_hs'].to(torch.float16)
del res['all_hs']
all_probe_hs = {layer_ix: all_probe_hs[:, save_ix, :] for save_ix, layer_ix in enumerate(layers_to_probe)}
gc.collect()

## Run probes

In [ ]:
clear_all_cuda_memory()

In [ ]:
"""
Run a quick grid search over hyperparmeters
"""
SKIP_FIRST_N = 8 # Skip first N tokens in each sample

def fit_lr(x_train, y_train, x_test, y_test, **lr_params):
    """
    Fit a probe
    """
    lr_model = sklearn.pipeline.Pipeline([
        # ('scaler', cuml.preprocessing.StandardScaler()),
        ('clf', cuml.linear_model.LogisticRegression(
            penalty = 'l2', # 1e-3 if model_prefix == 'gptoss20' else 1.0,
            max_iter = 5_000,
            fit_intercept = True,
            **lr_params
        ))
    ])
    lr_model.fit(x_train, y_train)
    accuracy = lr_model.score(x_test, y_test)
    y_test_pred = lr_model.predict(x_test)
    return lr_model, accuracy, y_test_pred

def get_probe_result(sample_df, layer_hs, roles_map, **lr_params):
    """
    Get probe results for a single layer and label combination

    Params:
        @sample_df: The sample-level df; with a column `sample_ix` indicating the token order of 0...T-1;
            the actual df may be shorter due to pre-filters
        @layer_hs: A tensor of probe hidden states for a layer, of T x D
        @roles_map: The mapping order of the roles; a dict {}

    Description:
        Trains only on content space for given roles
    """
    # Train/test split
    prompt_ix_train, prompt_ix_test = cuml.train_test_split(sample_df['prompt_ix'].unique(), test_size = 0.1, random_state = seed)
    train_df = sample_df[sample_df['prompt_ix'].isin(prompt_ix_train)]
    test_df = sample_df[sample_df['prompt_ix'].isin(prompt_ix_test)]

    # Get y labels
    role_labels_train_cp = cupy.asarray([roles_map[r] for r in train_df['role']])
    role_labels_test_cp = cupy.asarray([roles_map[r] for r in test_df['role']])

    # Get x labels
    x_train_cp = cupy.asarray(layer_hs[train_df['sample_ix'].tolist(), :].to(torch.float32).detach().cpu())
    x_test_cp = cupy.asarray(layer_hs[test_df['sample_ix'].tolist(), :].to(torch.float32).detach().cpu())
    
    uniq_train = np.unique(role_labels_train_cp.get())
    if len(uniq_train) < len(roles_map):
        raise Exception(f"Skipping mapping {roles_map}: missing roles in train", uniq_train)

    lr_model, test_acc, y_test_pred = fit_lr(x_train_cp, role_labels_train_cp, x_test_cp, role_labels_test_cp, **lr_params)

    # Classification metrics
    results_df =\
        test_df\
        .assign(pred = y_test_pred.tolist())\
        .assign(pred = lambda df: df['pred'].map({v: k for k, v in roles_map.items()}))\
        .assign(is_acc = lambda df: df['role'] == df['pred'])

    acc_by_role =\
        results_df\
        .groupby(['role', 'pred'], as_index = False)\
        .agg(count = ('sample_ix', 'count'))

    acc_by_pos =\
        results_df\
        .groupby('token_in_seg_ix', as_index = False)\
        .agg(count = ('sample_ix', 'count'), acc = ('is_acc', 'mean'))

    return {
        'probe': lr_model, 'acc': test_acc,
        'acc_by_role': acc_by_role, 'acc_by_pos': acc_by_pos
    }

def test_c(C):   
    test_roles = ['user', 'cot', 'assistant'] if model_is_reasoning else ['user', 'assistant']
    test_layer_ix = layers_to_probe[(len(layers_to_probe) - 1) // 2] 
    acc = get_probe_result(
        sample_df = probe_sample_df[(probe_sample_df['role'].isin(test_roles)) & (probe_sample_df['token_in_seg_ix'] >= SKIP_FIRST_N)].reset_index(drop = True),
        layer_hs = all_probe_hs[test_layer_ix],
        roles_map = {x: i for i, x in enumerate(test_roles)},
        C = C
    )['acc']
    return {'C': C, 'val_acc': acc}

hp_vals = [test_c(c_val) for c_val in tqdm([0.0001, 0.001, 0.01, 0.1, 1.0, 10.0])]
hp_vals

In [ ]:
"""
Run LRs
"""
all_role_combinations = [
    ('user', 'cot', 'assistant'),
    ('user', 'assistant', 'tool'),
    ('system', 'user', 'cot', 'assistant'),
    ('system', 'user', 'assistant', 'tool'),
    ('user', 'cot', 'assistant', 'tool'),
    ('system', 'user', 'cot', 'assistant', 'tool')
]
if not model_is_reasoning:
    all_role_combinations = [roles for roles in all_role_combinations if 'cot' not in roles]
    
all_role_combinations = [
    {
        'roles': list(roles),
        'roles_map': {x: i for i, x in enumerate(roles)},
        # Sample df for only those roles
        'sample_df': probe_sample_df.pipe(lambda df: df[(df['role'].isin(roles)) & (df['token_in_seg_ix'] >= SKIP_FIRST_N)]).reset_index(drop = True),
    }
    for roles in all_role_combinations
]

# clear_all_cuda_memory()

all_probes = []
for roles_dict in all_role_combinations:
    print(f"Training {roles_dict['roles']}")
    for layer_ix in layers_to_probe:
        probe_res = get_probe_result(
            sample_df = roles_dict['sample_df'],
            layer_hs = all_probe_hs[layer_ix],
            roles_map = roles_dict['roles_map'],
            C = 1e-3 if model_architecture == 'gptoss' else 1e-1
        )
        print(f"  Layer [{layer_ix}]: {probe_res['acc']:.2f}")

        all_probes.append({
            **probe_res,
            'layer_ix': layer_ix,
            'roles': roles_dict['roles'],
            'roles_map': roles_dict['roles_map'],
            'n_inputs': len(roles_dict['sample_df'])
        })

print(f"Num probes: {str(len(all_probes))}")
print(f"Probe layers:\n  {', '.join([str(x) for x in layers_to_probe])}")

In [ ]:
"""
Save probes + metrics
"""
def validate_accuracy_and_save(all_probes):
    print('Val accuracy by layer:')
    display(
        pd.DataFrame(all_probes)[['layer_ix', 'roles', 'acc']]\
            .assign(acc = lambda df: df['acc'].round(2), roles = lambda df: df['roles'].apply(lambda x: ','.join(([r[0] for r in x]))))\
            .pivot(index = 'layer_ix', columns = 'roles', values = 'acc')
    )

    print('Concatenating across layers and saving...')
    acc_by_role = pd.concat([p['acc_by_role'].assign(model = model_prefix, layer_ix = p['layer_ix'], roles = ','.join(p['roles'])) for p in all_probes], ignore_index = True)
    acc_by_pos =\
        pd.concat([p['acc_by_pos'].assign(model = model_prefix, layer_ix = p['layer_ix'], roles = ','.join(p['roles'])) for p in all_probes], ignore_index = True)\
        .assign(acc = lambda df: df['acc'].round(4))

    acc_by_role.to_csv(f'{ws}/experiments/role-analysis/probes/acc_by_role_{model_prefix}.csv', index = False)
    acc_by_pos.to_csv(f'{ws}/experiments/role-analysis/probes/acc_by_pos_{model_prefix}.csv', index = False)

    with open(f'{ws}/experiments/role-analysis/probes/{model_prefix}.pkl', 'wb') as f:
        pickle.dump(all_probes, f)

    print('Accuracy by role:')
    base_sums =\
        acc_by_role\
        .groupby(['role', 'layer_ix', 'roles'], as_index = False)\
        .agg(base_sum = ('count', 'sum'))

    acc_by_role =\
        acc_by_role\
        .pipe(lambda df: df[df['role'] == df['pred']])\
        .groupby(['role', 'layer_ix', 'roles'], as_index = False)\
        .agg(sum = ('count', 'sum'))\
        .merge(base_sums, on = ['layer_ix', 'roles', 'role'], how = 'inner')\
        .assign(acc = lambda df: df['sum']/df['base_sum'])\
        .pivot(index = ['roles', 'layer_ix'], columns = 'role', values = 'acc')\
        .round(2)

    display(acc_by_role)

    return True

validate_accuracy_and_save(all_probes)

# Generalization to conversations

## Load data

In [ ]:
"""
Validate IFT on the full chat template, which tokenizes the whole thing e2e. This should match the format from original messages.:
- `load_chat_template`: Overwrites the existing chat template in the tokenizer with one that better handles discrepancies.
    See docstring for the function for details. 
- `label_content_roles`: Takes an instruct-formatted text, then assigns each token to the correct roles.
"""
import importlib
import utils.role_assignments
import utils.role_templates
importlib.reload(utils.role_assignments)
label_content_roles = utils.role_assignments.label_content_roles
importlib.reload(utils.role_templates)
load_chat_template = utils.role_templates.load_chat_template

# Load custom chat templater
old_chat_template = tokenizer.chat_template
new_chat_template = load_chat_template(f'{ws}/utils/chat_templates', model_architecture)

def test_chat_template(tokenizer):
    s = tokenizer.apply_chat_template(
        [
            {'role': 'system', 'content': 'Test.'},
            {'role': 'user', 'content': 'Hi! I\'m a dog.'},
            {'role': 'assistant', 'content': '<think>The user is a dog!</think>Congrats!'},
            {'role': 'user', 'content': 'Thanks!'},
            {'role': 'assistant', 'content': '<think>Hmm, the user said thanks!</think>Anything else?'}
        ],
        tokenize = False,
        padding = 'max_length',
        truncation = True,
        max_length = 512,
        add_generation_prompt = False
    )
    print(s)
    return s

tokenizer.chat_template = old_chat_template
s = test_chat_template(tokenizer)
tokenizer.chat_template = new_chat_template
s = test_chat_template(tokenizer)

z =\
    pd.DataFrame({'input_ids': tokenizer(s)['input_ids']})\
    .assign(token = lambda df: tokenizer.convert_ids_to_tokens(df['input_ids']))\
    .assign(prompt_ix = 0, batch_ix = 0, sequence_ix = 0, token_ix = lambda df: df.index)

label_content_roles(model_architecture, z).tail(50)

In [ ]:
"""
Define opening text to append at the start of each role
"""
if model_architecture == 'gptoss':
    test_prefix = tokenizer.bos_token + render_single_message(
        model_architecture,
        'system', 
        textwrap.dedent(
        """
        You are ChatGPT, a large language model trained by OpenAI.
        Knowledge cutoff: 2024-06
        Current date: 2026-01-01

        Reasoning: medium

        # Valid channels: analysis, commentary, final. Channel must be included for every message.
        """).strip()
    )

elif model_architecture == 'olmo3':
    test_prefix = tokenizer.bos_token + render_single_message(
            model_architecture,
            'system', 
            textwrap.dedent(
            """
            You are OLMo, a helpful function-calling AI assistant built by Ai2. Your date cutoff is November 2024, and your model weights are available at https://huggingface.co/allenai. You do not currently have access to any functions. <functions></functions>
            """).strip()
        )
    
elif model_architecture in ['glm46v', 'glm4vmoe']:
    test_prefix = """[gMASK]<sop>"""
        
elif model_architecture == 'apriel':
    test_prefix = tokenizer.bos_token + render_single_message(
        model_architecture,
        'system',
        textwrap.dedent(
        """
        You are a thoughtful, systematic AI assistant from ServiceNow Language Models (SLAM) lab. Analyze each question carefully, present your reasoning step-by-step, then provide the final response after the marker [BEGIN FINAL RESPONSE].
        """
        ).strip()
    )

else:
    raise ValueError('Missing model architecture')

print(test_prefix)

In [ ]:
"""
Validate with prefix + generation ability
"""
def test():
    conv = test_prefix + tokenizer.apply_chat_template(
        [
            {'role': 'user', 'content': 'Hi stinky pupper'}    
        ],
        tokenize = False,
        add_generation_prompt = True
    )

    inputs = tokenizer(conv, return_tensors = 'pt')
    gen_ids = model.generate(inputs['input_ids'].to(main_device), attention_mask = inputs['attention_mask'].to(main_device), max_new_tokens = 100, do_sample = False)
    print(tokenizer.decode(gen_ids)[0])

    df = pd.DataFrame({'token_id': gen_ids[0].tolist(), 'token': tokenizer.convert_ids_to_tokens(gen_ids[0].tolist()),})
    return df

display(test())

In [ ]:
"""
Import conversations
"""
def filter_convs(user_queries_df, max_samples):
    print(f"Starting length: {len(user_queries_df['conv_id'].unique())} convs")

    # 1. Remove convs with any missing responses
    convs_with_missing = user_queries_df[user_queries_df[['user_query', 'cot', 'assistant']].isna().any(axis = 1)]['conv_id'].unique()
    user_queries_df = user_queries_df[~user_queries_df['conv_id'].isin(convs_with_missing)]
    print(f"After dropping missing: {len(user_queries_df['conv_id'].unique())} convs")

    # 2. Filter out super long convs + convs with any message <= conv_min_length (to avoid issues with flag_message_type fn)
    user_queries_df =\
        user_queries_df\
        .assign(total_len = lambda df:
            df.groupby('conv_id')['user_query'].transform(lambda x: x.str.len().sum()) +
            df.groupby('conv_id')['cot'].transform(lambda x: x.str.len().sum()) +
            df.groupby('conv_id')['assistant'].transform(lambda x: x.str.len().sum())
        )\
        .assign(min_len = lambda df: df[['user_query', 'cot', 'assistant']].apply(lambda x: x.str.len()).min(axis=1))\
        .assign(conv_min_len = lambda df: df.groupby('conv_id')['min_len'].transform('min'))\
        .assign(conv_min_user_len = lambda df: df.groupby('conv_id')['user_query'].transform(lambda x: x.str.len().min()))\
        .query('total_len < 16000 and conv_min_len >= 20 and conv_min_user_len >= 100')\
        .drop(columns = ['total_len', 'min_len', 'conv_min_len', 'conv_min_user_len'])

    print(f"After dropping long convs: {len(user_queries_df['conv_id'].unique())} convs")

    # 3. Filter out convs where any message is a substring of another message
    #    (this causes dupe issues with the flag_message_type fn, which does not handle tokens in multiple string contexts)
    def has_substring_messages(group):
        contents = group['user_query'].tolist() + group['cot'].tolist() + group['assistant'].tolist()
        for i, a in enumerate(contents):
            for j, b in enumerate(contents):
                if i != j and a in b:
                    return True
        return False

    bad_convs = user_queries_df.groupby('conv_id').filter(has_substring_messages)['conv_id'].unique()
    user_queries_df = user_queries_df.pipe(lambda df: df[~df['conv_id'].isin(bad_convs)])
    print(f"After dropping substr: {len(user_queries_df['conv_id'].unique())} convs")

    # Combine into conv-level df
    convs_df =\
        user_queries_df\
        .sort_values(['conv_id', 'user_query_ix'])\
        .assign(conv_id = lambda df: df.groupby('conv_id', sort = False).ngroup())\
        .groupby('conv_id')\
        .apply(lambda g: pd.Series({
            'dataset': g['dataset'].values[0],
            'messages': [
                msg
                for _, row in g.iterrows()
                for msg in [
                    {'role': 'user', 'content': row['user_query']},
                    {'role': 'cot', 'content': row['cot']},
                    {'role': 'assistant', 'content': row['assistant']}
                ]
            ]
        }))\
        .reset_index()\
        [['conv_id', 'dataset', 'messages']]\
        .sample(n = max_samples, random_state = seed)\
        .assign(conv_id = lambda df: range(len(df)))

    print(f"Final: {len(convs_df['conv_id'].unique())} convs")

    # messages_df =\
    #     convs_df\
    #     .explode('messages')\
    #     .assign(message_ix = lambda df: df.groupby('conv_id').cumcount())\
    #     .assign(
    #         role = lambda df: df['messages'].apply(lambda x: x['role']),
    #         content = lambda df: df['messages'].apply(lambda x: x['content'])
    #     )\
    #     .drop(columns = 'messages')\
    #     .reset_index(drop = True)

    convs = convs_df['messages'].tolist()
    return convs, convs_df

convs, convs_df = filter_convs(
    user_queries_df = pd.read_csv(f'{ws}/experiments/role-analysis/convs/{model_prefix}.csv'),
    max_samples = 40
)

display(convs_df.head(5))
print(f"Total convs: {len(convs_df)}")

In [ ]:
"""
Prep untagged conversations
"""
all_convs_by_type = {}

# Prep conversations
def prep_conv(conv):
    return test_prefix + '\n' + '\n'.join([x['content'] for x in conv])

all_convs_by_type['untagged'] = [prep_conv(conv) for conv in convs]
print(all_convs_by_type['untagged'][0])

# Find max input length
max_input_length =\
    tokenizer(all_convs_by_type['untagged'], padding = True, truncation = True, max_length = 1024 * 6, return_tensors = 'pt')\
    ['attention_mask'].sum(dim = 1).max().item()

print(f"Max input length: {max_input_length}")
print(f"Untagged convs: {len(all_convs_by_type['untagged'])}")

In [ ]:
"""
Prep tagged conversations
"""
def fold_cot_into_final(convs):
    """Fold CoT into the following assistant message as a <think></think> tag."""
    result = []
    for conv in convs:
        new_conv = []
        it = iter(enumerate(conv))
        for i, msg in it:
            if msg['role'] == 'cot':
                if i + 1 >= len(conv) or conv[i + 1]['role'] != 'assistant':
                    raise ValueError("cot must be followed by assistant")
                _, next_msg = next(it)
                new_conv.append({'role': 'assistant', 'content': f"<think>{msg['content']}</think>{next_msg['content']}"})
            else:
                new_conv.append({'role': msg['role'], 'content': msg['content']})
        result.append(new_conv)
    return result

final_convs = fold_cot_into_final(convs)
all_convs_by_type['tagged'] = [
    test_prefix + x
    for x in tokenizer.apply_chat_template(final_convs, tokenize = False, add_generation_prompt = False)
]
print(all_convs_by_type['tagged'][0])

# Find max input length
max_input_length =\
    tokenizer(all_convs_by_type['tagged'], padding = True, truncation = True, max_length = 1024 * 8, return_tensors = 'pt')\
    ['attention_mask'].sum(dim = 1).max().item()

print(f"Max input length: {max_input_length}")
print(f"Tagged convs: {len(all_convs_by_type['tagged'])}")

In [ ]:
"""
Prep mistagged conversations
"""
all_convs_by_type['user_tagged'] = [render_single_message(model_architecture, 'user', prep_conv(conv)) for conv in convs]
print(all_convs_by_type['user_tagged'][0])

all_convs_by_type['tool_tagged'] = [render_single_message(model_architecture, 'tool', prep_conv(conv)) for conv in convs]
print(all_convs_by_type['tool_tagged'][0])

## Run passes

In [ ]:
"""
Run forward passes
"""
# Get input df
input_convs_df =\
    pd.concat([
        pd.DataFrame({'convs': v, 'conv_type': conv_type})\
            .assign(conv_id = lambda df: range(0, len(df)))\
            .merge(convs_df, how = 'inner', on = 'conv_id')
        for conv_type, v in all_convs_by_type.items()
    ], ignore_index = True)\
    .assign(prompt_ix = lambda df: list(range(0, len(df))))
    
display(input_convs_df)

convs_dl = DataLoader(
    ReconstructableTextDataset(input_convs_df['convs'].tolist(), tokenizer, max_length = max_input_length, prompt_ix = list(range(0, len(input_convs_df)))),
    batch_size = 8,
    shuffle = False,
    collate_fn = stack_collate
)

convs_outputs = run_and_export_states(model, convs_dl, layers_to_keep_acts = layers_to_probe)

In [ ]:
"""
Label messages
"""
import utils.substring_assignments
importlib.reload(utils.substring_assignments)
from utils.substring_assignments import flag_message_types

sample_dfs_by_conv = [group for _, group in convs_outputs['sample_df'].groupby('prompt_ix', sort = True)]
metadata_by_conv = input_convs_df.to_dict('records')

all_res = []

# Iterate through (input metadata, token-level sample df) pairs
for conv_metadata, sample_df_for_conv in tqdm(zip(metadata_by_conv, sample_dfs_by_conv)):

    content_spans = [msg['content'] for msg in conv_metadata['messages']]
    content_roles = [msg['role'] for msg in conv_metadata['messages']]
    
    try:
        # Note: set the last arg of flag_message_types for Olmo3/other models whose CoTs quote the FULL user message (one base_message is a subset of another)
        # Setting the flag causes it to default to the first matching BMT (which should be user)
        res = \
            flag_message_types(sample_df_for_conv, content_spans, any(x in model_prefix for x in ['olmo', 'glm']))\
            .merge(
                pd.DataFrame({'role': content_roles, 'base_message_ix': range(len(content_spans))}),
                on = 'base_message_ix',
                how = 'left'
            )
    except Exception as e:
        print(e)
        continue
    
    all_res.append(res)

display(pd.concat(all_res).pipe(lambda df: df[df['role'].isna()]))
        
sample_df_labeled =\
    pd.concat(all_res).reset_index(drop = True)\
    .assign(sample_ix = lambda df: range(0, len(df)))\
    .assign(token_in_prompt_ix = lambda df: df.groupby(['prompt_ix']).cumcount())

display(sample_df_labeled.pipe(lambda df: df[~df['role'].isna()]))

display(
    sample_df_labeled\
    .pipe(lambda df: df[df['prompt_ix'] <= 5])\
    .groupby(['prompt_ix', 'base_message', 'role'], as_index = False)\
    .agg(combined_text = ('token', ''.join))\
    .assign(eot = lambda df: df['combined_text'].str[-30:])
)

display(
    sample_df_labeled\
    .pipe(lambda df: df[~df['role'].isna()])\
    .groupby(['prompt_ix', 'role'], as_index = False)\
    .agg(count = ('sample_ix', 'count'))\
    .pivot(index = ['prompt_ix'], columns = 'role', values = 'count')\
    .head(20)
)

In [ ]:
"""
Clean up conversation pre_mlp_hs
"""
convs_hs = convs_outputs['all_hs'].to(torch.float16)
convs_hs = {layer_ix: convs_hs[:, save_ix, :] for save_ix, layer_ix in enumerate(layers_to_probe)} # Match layers_to_keep_act
print(convs_hs[0].shape)
print(len(sample_df_labeled))
gc.collect()

## Run projections

In [ ]:
"""
Run test projections for single layer
"""
def run_projections(valid_sample_df, layer_hs, probe):
    """
    Run probe-level projections
    
    Params:
        @valid_sample_df: A sample-level df with columns `sample_ix` (1... T - 1), `sample_ix`.
            Can be shorter than full T - 1 due to pre-filters, as long as sample_ix corresponds to the full length.
        @layer_hs: A tensor of size T x D for the layer to project.
        @probe: The probe dict with keys `probe` (the trained model) and `roles` (the roles list)
    
    Returns:
        A df at (sample_ix, role_space) level with cols `sample_ix`, `role_space`, `prob`
    """
    x_cp = cupy.asarray(layer_hs[valid_sample_df['sample_ix'].tolist(), :])
    y_cp = probe['probe'].predict_proba(x_cp).round(8)

    proj_results = pd.DataFrame(cupy.asnumpy(y_cp), columns = probe['roles'])
    if len(proj_results) != len(valid_sample_df):
        raise Exception("Error!")

    role_df =\
        pd.concat([
            proj_results.reset_index(drop = True),
            valid_sample_df[['sample_ix']].reset_index(drop = True)
        ], axis = 1)\
        .melt(id_vars = ['sample_ix'], var_name = 'role_space', value_name = 'prob')\
        .reset_index(drop = True)

    return role_df

# top_train_layer =\
#     pd.DataFrame(all_probes)\
#     .assign(probe_ix = lambda df: range(0, len(df)))\
#     .pipe(lambda df: df[df['roles'].apply(lambda r: sorted(r) == sorted(test_roles))])\
#     .pipe(lambda df: df.nlargest(1, 'acc')).to_dict('records')[0]
# print(f"Top layer: {top_train_layer['layer_ix']}")
# test_layer = top_train_layer['layer_ix']

test_roles = ['system', 'user', 'cot', 'assistant'] if model_is_reasoning else ['system', 'user', 'assistant']
test_layer_ix = 16

test_projections =\
    run_projections(
        valid_sample_df = sample_df_labeled.pipe(lambda df: df[(~df['role'].isna()) & (df['role'].isin(test_roles))]),
        layer_hs = convs_hs[test_layer_ix],
        probe = [x for x in all_probes if sorted(x['roles']) == sorted(test_roles) and x['layer_ix'] == test_layer_ix][0]
    )\
    .merge(sample_df_labeled[['prompt_ix', 'sample_ix', 'token_in_prompt_ix', 'token', 'role']], how = 'inner', on = ['sample_ix'])\
    .merge(input_convs_df[['prompt_ix', 'dataset', 'conv_id', 'conv_type']], how = 'inner', on = 'prompt_ix', validate = 'many_to_one')


# Now aggregate up to (prompt, role, role_space) level
prompt_x_role_x_role_space_projections =\
    test_projections\
    .groupby(['conv_type', 'prompt_ix', 'role', 'role_space'], as_index = False)\
    .agg(
        combined_text = ('token', ''.join),
        mean_prob = ('prob', lambda x: np.mean(x).round(3))    
    )\
    .assign(eot = lambda df: df['combined_text'].str[-30:])\

display(prompt_x_role_x_role_space_projections.head(12))

display(
    prompt_x_role_x_role_space_projections\
    .groupby(['conv_type', 'role_space', 'role'], as_index = False)\
    .agg(mean_prob = ('mean_prob', 'mean'))\
    .pivot(index = ['conv_type', 'role'], columns = 'role_space', values = 'mean_prob')
)

In [ ]:
print(test_projections\
    .pipe(lambda df: df[df['prompt_ix'] == 30])\
    .pipe(lambda df: df[df['role'] == 'cot'])\
    .groupby('role_space')\
    .agg(combined_text = ('token', ''.join))
    ['combined_text'].tolist()[0]
)
print('-----------')
print(test_projections\
    # .pipe(lambda df: df[df['conv_type'] == 'tagged'])\
    .pipe(lambda df: df[df['prompt_ix'] == len(convs) + 30])\
    .pipe(lambda df: df[df['role'] == 'cot'])\
    .groupby('role_space')\
    .agg(combined_text = ('token', ''.join))\
    ['combined_text'].tolist()[0]
)

In [ ]:
"""
All-layer projections
"""
clear_all_cuda_memory()
gc.collect()

all_projs = [
    run_projections(
        valid_sample_df = sample_df_labeled.pipe(lambda df: df[(~df['role'].isna()) & (df['role'].isin(probe['roles']))]),
        layer_hs = convs_hs[probe['layer_ix']],
        probe = probe
    ).assign(layer_ix = probe['layer_ix'], roles = ''.join([x[0] for x in probe['roles']]))

    for probe in tqdm(all_probes)
]

all_projs_df =\
    pd.concat(all_projs, ignore_index = True)\
    .merge(sample_df_labeled[['prompt_ix', 'sample_ix', 'token_in_prompt_ix', 'token', 'role']], how = 'inner', on = ['sample_ix'])\
    .merge(input_convs_df[['prompt_ix', 'dataset', 'conv_id', 'conv_type']], how = 'inner', on = 'prompt_ix', validate = 'many_to_one')

all_projs_df

In [ ]:
"""
Validation
"""
out =\
    all_projs_df\
    .pipe(lambda df: df[df['role_space'] == df['role']])\
    .drop(columns = 'role_space')\
    .rename(columns = {'prob': 'acc'})\
    .groupby(['conv_type', 'roles', 'layer_ix', 'role', 'prompt_ix'], as_index = False)\
    .agg(mean_acc = ('acc', 'mean'))\
    .groupby(['conv_type', 'roles', 'layer_ix', 'role'], as_index = False)\
    .agg(mean_acc = ('mean_acc', 'mean'))\
    .assign(
        role = lambda df: np.where(df['role'] == 'assistant', 'asst', df['role'])
    )\
    .pivot(index = ['conv_type', 'layer_ix'], columns = ['roles', 'role'], values = 'mean_acc')

def color_thresholds(v):
    if pd.isna(v):
        return ""
    if v > 0.9:
        return "background-color: #00c853; color: black;" # green
    elif v > 0.75:
        return "background-color: #aeea00; color: black;" # green-yellow
    elif v > 0.6:
        return "background-color: #ffeb3b; color: black;" # yellow
    return ""

col_groups = out.columns.get_level_values(0)
group_starts = [0] + [i for i in range(1, len(col_groups)) if col_groups[i] != col_groups[i-1]]
divider_styles = []
for i in group_starts:
    divider_styles += [
        {"selector": f"td.col{i}", "props": [("border-left", "3px solid #ddd")]},
        {"selector": f"th.col_heading.col{i}", "props": [("border-left", "3px solid #ddd")]},
    ]

display(out.style.format("{:.2f}").map(color_thresholds).set_table_styles(divider_styles, overwrite = False))

In [ ]:
"""
Actual accuracy (instead of averaged roleness)
"""
# Take argmax role_space per token for each (roles, layer_ix)
max_role_space_idxs =\
    all_projs_df\
    .groupby(['conv_type', 'roles', 'layer_ix', 'sample_ix'], sort = False)['prob']\
    .idxmax()

pred_df =\
    all_projs_df.loc[max_role_space_idxs, ['conv_type', 'roles', 'layer_ix', 'sample_ix', 'prompt_ix', 'role', 'role_space']]\
    .rename(columns = {'role_space': 'pred_role'})\
    .assign(is_correct = lambda df: (df['pred_role'] == df['role']).astype(float))

# Only score tokens whose true role is IN the probe label set for that 'roles' group. (ow roles like 'tool' will be forced-incorrect when not in labelset.)
roles_padded = ',' + pred_df['roles'] + ','
possible_roles = ['system', 'user', 'cot', 'assistant', 'tool']

mask = np.zeros(len(pred_df), dtype=bool)
for r in possible_roles:
    mask |= (pred_df['role'].eq(r) & roles_padded.str.contains(f',{r},', regex = False))

pred_df_scored = pred_df[mask].copy()

out_argmax = (
    pred_df_scored
    .groupby(['conv_type', 'roles', 'layer_ix', 'role', 'prompt_ix'], as_index=False)
    .agg(acc = ('is_correct', 'mean'))
    .groupby(['conv_type', 'roles', 'layer_ix', 'role'], as_index=False)
    .agg(mean_acc = ('acc', 'mean'))
    .assign(
        roles = lambda df: df['roles'].apply(lambda x: ''.join([c[0] for c in x.split(',')])),
        role = lambda df: np.where(df['role'] == 'assistant', 'asst', df['role'])
    )
    .pivot(index = ['conv_type', 'layer_ix'], columns = ['roles', 'role'], values = 'mean_acc')
)

display(out_argmax.style.format("{:.2f}").map(color_thresholds).set_table_styles(divider_styles, overwrite = False))

In [ ]:
"""
Save and export
"""
all_projs_df.to_csv(f'{ws}/experiments/role-analysis/probes/all_conv_projs_{model_prefix}.csv', index = False)

## Wrong convs

In [ ]:
"""
Prep wrong-convs
"""
all_model_files = [
    'kimik2'
]

for m in all_model_files:
    m_df = pd.read_csv(f'{ws}/experiments/role-analysis/convs/{m}.csv')

alt_convs, alt_convs_df = filter_convs(m_df, max_samples = 20)

final_alt_convs = fold_cot_into_final(alt_convs)
all_wrong_convs_tagged = [
    test_prefix + x
    for x in tokenizer.apply_chat_template(final_alt_convs, tokenize = False, add_generation_prompt = False)
]
print(all_wrong_convs_tagged[0])

# Find max input length
max_alt_input_length =\
    tokenizer(all_wrong_convs_tagged, padding = True, truncation = True, max_length = 1024 * 8, return_tensors = 'pt')\
    ['attention_mask'].sum(dim = 1).max().item()

print(f"Max input length: {max_alt_input_length}")
print(f"Tagged convs: {len(all_wrong_convs_tagged)}")

In [ ]:
"""
Run forward passes
"""
# Get input df
input_alt_convs_df =\
    pd.concat([
        pd.DataFrame({'convs': v, 'conv_type': conv_type})\
            .assign(conv_id = lambda df: range(0, len(df)))\
            .merge(alt_convs_df, how = 'inner', on = 'conv_id')
        for conv_type, v in {'alt_tagged': all_wrong_convs_tagged}.items()
    ], ignore_index = True)\
    .assign(prompt_ix = lambda df: list(range(0, len(df))))
    
display(input_alt_convs_df)

alt_convs_dl = DataLoader(
    ReconstructableTextDataset(input_alt_convs_df['convs'].tolist(), tokenizer, max_length = max_input_length, prompt_ix = list(range(0, len(input_alt_convs_df)))),
    batch_size = 8,
    shuffle = False,
    collate_fn = stack_collate
)

alt_convs_outputs = run_and_export_states(model, alt_convs_dl, layers_to_keep_acts = layers_to_probe)

In [ ]:
"""
Label messages
"""
import utils.substring_assignments
importlib.reload(utils.substring_assignments)
from utils.substring_assignments import flag_message_types

alt_sample_dfs_by_conv = [group for _, group in alt_convs_outputs['sample_df'].groupby('prompt_ix', sort = True)]
alt_metadata_by_conv = input_alt_convs_df.to_dict('records')

all_res = []

# Iterate through (input metadata, token-level sample df) pairs
for conv_metadata, sample_df_for_conv in tqdm(zip(alt_metadata_by_conv, alt_sample_dfs_by_conv)):

    content_spans = [msg['content'] for msg in conv_metadata['messages']]
    content_roles = [msg['role'] for msg in conv_metadata['messages']]
    
    try:
        # Note: set the last arg of flag_message_types for Olmo3/other models whose CoTs quote the FULL user message (one base_message is a subset of another)
        # Setting the flag causes it to default to the first matching BMT (which should be user)
        res = \
            flag_message_types(sample_df_for_conv, content_spans, any(x in model_prefix for x in ['olmo', 'glm']))\
            .merge(
                pd.DataFrame({'role': content_roles, 'base_message_ix': range(len(content_spans))}),
                on = 'base_message_ix',
                how = 'left'
            )
    except Exception as e:
        print(e)
        continue
    
    all_res.append(res)

display(pd.concat(all_res).pipe(lambda df: df[df['role'].isna()]))
        
alt_sample_df_labeled =\
    pd.concat(all_res).reset_index(drop = True)\
    .assign(sample_ix = lambda df: range(0, len(df)))\
    .assign(token_in_prompt_ix = lambda df: df.groupby(['prompt_ix']).cumcount())

display(alt_sample_df_labeled.pipe(lambda df: df[~df['role'].isna()]))

display(
    alt_sample_df_labeled\
    .pipe(lambda df: df[df['prompt_ix'] <= 5])\
    .groupby(['prompt_ix', 'base_message', 'role'], as_index = False)\
    .agg(combined_text = ('token', ''.join))\
    .assign(eot = lambda df: df['combined_text'].str[-30:])
)

display(
    alt_sample_df_labeled\
    .pipe(lambda df: df[~df['role'].isna()])\
    .groupby(['prompt_ix', 'role'], as_index = False)\
    .agg(count = ('sample_ix', 'count'))\
    .pivot(index = ['prompt_ix'], columns = 'role', values = 'count')\
    .head(20)
)

In [ ]:
"""
Clean up conversation pre_mlp_hs
"""
alt_convs_hs = alt_convs_outputs['all_hs'].to(torch.float16)
alt_convs_hs = {layer_ix: alt_convs_hs[:, save_ix, :] for save_ix, layer_ix in enumerate(layers_to_probe)} # Match layers_to_keep_act
print(alt_convs_hs[0].shape)
print(len(alt_sample_df_labeled))
gc.collect()

In [ ]:
"""
All-layer projections
"""
clear_all_cuda_memory()
gc.collect()

alt_projs = [
    run_projections(
        valid_sample_df = alt_sample_df_labeled.pipe(lambda df: df[(~df['role'].isna()) & (df['role'].isin(probe['roles']))]),
        layer_hs = alt_convs_hs[probe['layer_ix']],
        probe = probe
    ).assign(layer_ix = probe['layer_ix'], roles = ''.join([x[0] for x in probe['roles']]))

    for probe in tqdm(all_probes)
]

alt_projs_df =\
    pd.concat(all_projs, ignore_index = True)\
    .merge(alt_sample_df_labeled[['prompt_ix', 'sample_ix', 'token_in_prompt_ix', 'token', 'role']], how = 'inner', on = ['sample_ix'])\
    .merge(input_alt_convs_df[['prompt_ix', 'dataset', 'conv_id', 'conv_type']], how = 'inner', on = 'prompt_ix', validate = 'many_to_one')

alt_projs_df

In [ ]:
"""
Validation
"""
out =\
    alt_projs_df\
    .pipe(lambda df: df[df['role_space'] == df['role']])\
    .drop(columns = 'role_space')\
    .rename(columns = {'prob': 'acc'})\
    .groupby(['conv_type', 'roles', 'layer_ix', 'role', 'prompt_ix'], as_index = False)\
    .agg(mean_acc = ('acc', 'mean'))\
    .groupby(['conv_type', 'roles', 'layer_ix', 'role'], as_index = False)\
    .agg(mean_acc = ('mean_acc', 'mean'))\
    .assign(
        role = lambda df: np.where(df['role'] == 'assistant', 'asst', df['role'])
    )\
    .pivot(index = ['conv_type', 'layer_ix'], columns = ['roles', 'role'], values = 'mean_acc')

def color_thresholds(v):
    if pd.isna(v):
        return ""
    if v > 0.9:
        return "background-color: #00c853; color: black;" # green
    elif v > 0.75:
        return "background-color: #aeea00; color: black;" # green-yellow
    elif v > 0.6:
        return "background-color: #ffeb3b; color: black;" # yellow
    return ""

col_groups = out.columns.get_level_values(0)
group_starts = [0] + [i for i in range(1, len(col_groups)) if col_groups[i] != col_groups[i-1]]
divider_styles = []
for i in group_starts:
    divider_styles += [
        {"selector": f"td.col{i}", "props": [("border-left", "3px solid #ddd")]},
        {"selector": f"th.col_heading.col{i}", "props": [("border-left", "3px solid #ddd")]},
    ]

display(out.style.format("{:.2f}").map(color_thresholds).set_table_styles(divider_styles, overwrite = False))

# Tomato example

In [ ]:
"""
Define append prefix
"""
tomato_prefix = ''# tokenizer.bos_token

tomato_prefix

In [ ]:
"""
Format tomato-style conversations
"""
import yaml

def load_messages(yaml_path, model_key):
    """Loads system-like, assistant-like, user-like, etc. messages"""
    with open(yaml_path, 'r', encoding = 'utf-8') as f:
        data = yaml.safe_load(f)
    return [x['content'] for x in data[model_key]]

base_message_types = ['system', 'user', 'cot', 'assistant', 'user', 'cot', 'assistant']
base_messages = load_messages(f"{ws}/experiments/role-analysis/prompts/standard-conversations.yaml", 'gptoss20')

tomato_prompts = {}

tomato_prompts['basic_no_format'] = tomato_prefix + '\n'.join(base_messages)

tomato_prompts['everything_in_user_tags'] = tomato_prefix + tokenizer.apply_chat_template(
    [{'role': 'user', 'content': '\n'.join(base_messages)}],
    tokenize = False, add_generation_prompt = False
)

tomato_prompts['proper_tags'] = tomato_prefix + tokenizer.apply_chat_template(
    [
        {'role': 'system', 'content': base_messages[0]},
        {'role': 'user', 'content': base_messages[1]},
        {'role': 'assistant', 'content': f"<think>{base_messages[2]}</think>{base_messages[3]}"},
        {'role': 'user', 'content': base_messages[4]},
        {'role': 'assistant', 'content': f"<tool_call>{base_messages[5]}<tool_call>{base_messages[6]}"}
    ],
    tokenize = False, add_generation_prompt = False
)

print(tomato_prompts['proper_tags'])

tomato_input_df =\
    pd.DataFrame({
        'prompt': [p for _, p in tomato_prompts.items()],
        'prompt_key': [pk for pk, _ in tomato_prompts.items()]
    })\
    .assign(prompt_ix = lambda df: list(range(0, len(df))))

In [ ]:
"""
Run!!
"""
tomato_dl = DataLoader(
    ReconstructableTextDataset(tomato_input_df['prompt'].tolist(), tokenizer, max_length = 512 * 4, prompt_ix = tomato_input_df['prompt_ix'].tolist()),
    batch_size = 16,
    shuffle = False,
    collate_fn = stack_collate
)

tomato_outputs = run_and_export_states(model, tomato_dl, layers_to_keep_acts = layers_to_probe)

tomato_hs = tomato_outputs['all_hs'].to(torch.float16)
tomato_hs = {layer_ix: tomato_hs[:, save_ix, :] for save_ix, layer_ix in enumerate(layers_to_probe)} # Match layers_to_keep_act
print(tomato_hs[0].shape)
gc.collect()

In [ ]:
"""
Ensure tokens/section is reasonable; map to base_message_type
"""
tomato_sample_df_labeled =\
    flag_message_types(tomato_outputs['sample_df'], base_messages)\
    .assign(sample_ix = lambda df: range(0, len(df)))\
    .assign(token_in_prompt_ix = lambda df: df.groupby(['prompt_ix']).cumcount())\
    .merge(tomato_input_df, on = 'prompt_ix', how = 'inner')\
    .merge(
        pd.DataFrame({'base_message_ix': list(range(len(base_message_types))), 'base_message_type': base_message_types}),
        on = 'base_message_ix',
        how = 'inner'
    )

tomato_sample_df_labeled\
    .groupby(['prompt_ix', 'prompt_key', 'base_message_ix', 'base_message_type', 'base_message'], as_index = False)\
    .agg(
        count = ('token', 'count'),
        combined_text = ('token', ''.join)
    )

In [ ]:
gc.collect()

tomato_projs = [
    run_projections(
        valid_sample_df = tomato_sample_df_labeled.pipe(lambda df: df[(~df['base_message_type'].isna()) & (df['base_message_type'].isin(probe['roles']))]),
        layer_hs = tomato_hs[probe['layer_ix']],
        probe = probe
    ).assign(layer_ix = probe['layer_ix'], roles = ''.join([x[0] for x in probe['roles']]))

    for probe in tqdm(all_probes)
]

tomato_projs_df =\
    pd.concat(tomato_projs, ignore_index = True)\
    .merge(
        tomato_sample_df_labeled[['sample_ix', 'prompt_ix', 'prompt_key', 'base_message_ix', 'base_message_type', 'token_in_prompt_ix', 'token']],
        how = 'inner',
        on = ['sample_ix']
    )

tomato_projs_df

In [ ]:
tomato_projs_df\
    .groupby(['prompt_ix', 'prompt_key', 'role_space', 'layer_ix', 'roles', 'sample_ix'], as_index = False)\
    .agg(count = ('sample_ix', 'count'))

In [ ]:
import plotly.express as px

facet_order = ['system', 'user', 'cot', 'assistant']
tomato_projs_df_single = tomato_projs_df.pipe(lambda df: df[(df['roles'] == 'suca') & (df['layer_ix'] == 12)])

all_message_types = tomato_projs_df['base_message_type'].unique()
color_map = {msg_type: px.colors.qualitative.Plotly[i % len(px.colors.qualitative.Plotly)] for i, msg_type in enumerate(all_message_types)}

group_cols = ['prompt_ix','prompt_key','base_message_type','role_space']

smooth_df = (
    tomato_projs_df_single
    .sort_values(group_cols + ['sample_ix'])
    .assign(
        prob_sma = lambda df: (
            df.groupby(group_cols)['prob']
              .rolling(window=10, min_periods=1).mean()
              .reset_index(level=list(range(len(group_cols))), drop=True)
        ),
        prob_ewma = lambda df: (
            df.groupby(group_cols)['prob']
              .ewm(alpha=.5, min_periods=1).mean()
              .reset_index(level=list(range(len(group_cols))), drop=True)
        ),
    )
)

display(smooth_df)

for this_prompt_key in smooth_df['prompt_key'].unique().tolist():
    this_df = smooth_df.pipe(lambda df: df[df['prompt_key'] == this_prompt_key])
    fig = px.scatter(
        this_df, x = 'token_in_prompt_ix', y = 'prob',
        facet_row = 'role_space',
        color = 'base_message_type',
        color_discrete_map = color_map,
        category_orders = {
            'role_space': facet_order,
            'base_message_type': ['system', 'user', 'cot', 'assistant', 'other']
        },
        hover_name = 'token',
        hover_data = {
            'prob': ':.3f'
        },
        # markers = True,
        title = f'prompt = {this_prompt_key}',
        labels = {
            'token_in_prompt_ix': 'Token Index',
            'prob': 'Prob',
            'prob_smoothed': 'Smoothed Prob',
            'role_space': 'role'
        }
    ).update_yaxes(
        range = [0, 1],
        side = 'left'
    ).update_layout(height = 500, width = 800)

    def pretty(a):
        a.update(
            text = a.text.split('=')[-1],
            x = 0.5, xanchor = "center",
            y = a.y + 0.08,
            textangle = 0,
            font = dict(size = 12),
            bgcolor = 'rgba(255, 255, 255, 0.9)',
            showarrow = False
        )

    fig.for_each_annotation(pretty)
    fig.update_yaxes(title_text = None)
    fig.show()

In [ ]:
tomato_projs_df.to_csv(f"{ws}/experiments/role-analysis/projections/tomato-role-projections-{model_prefix}.csv", index = False)